Extract the 10 days weather from san Francisco from weather.com.

inspect with the console

write code to target the right elements of the text

For each day, you need to store:

The day of the week (Tuesday)

The date (28/09/2020)

A short description of the conditions (Clear early then increasing cloudiness after midnight. Low 41F. Winds light and variable)

The temperature low and high, with a function of your own to convert into Celsius

For each element you scrape, The name of the item you targetted (ex: DailyContent--daypartDate--3MM0J)

Save all of this into a Pandas dataframe

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [6]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep


url = "https://weather.com/weather/tenday/l/San+Francisco+CA?canonicalCityId=dfdaba8cbe3a4d12a8796e1f7b1ccc7174b4b0a2d5ddb1c8566ae9f154fa638c"
path = "chromedriver.exe"

driver = webdriver.Chrome(path)

driver.get(url)


In [7]:
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
div = soup.find_all('div', id='WxuDailyCard-main-a43097e1-49d7-4df7-9d1a-334b29628263')
text = div[0].text
text

'10 Day Weather-San Francisco, CAAs of 9:27 am PSTToday58°/50°Partly CloudyPartly CloudyRain7%WindN 9 mphArrow UpTue 07 | Day58°Partly CloudyRain7%WindN 9 mphPartly cloudy skies. High 58F. Winds N at 5 to 10 mph.HumidityHumidity79%UV LevelUV Index2 of 10Sun RiseSunrise7:12 amSunsetSunset4:50 pmTue 07 | Night50°Mostly Cloudy NightRain17%WindW 5 mphPartly cloudy early followed by cloudy skies overnight. Expect mist and reduced visibilities at times. Low near 50F. Winds light and variable.HumidityHumidity95%UV LevelUV Index0 of 10Moon RiseMoonrise10:54 amMoon Phase - Day 4Waxing CrescentMoon SetMoonset8:41 pmWed 0855°/47°CloudyCloudyRain21%WindW 13 mphArrow DownWed 08 | Day55°CloudyRain21%WindW 13 mphOvercast. High around 55F. Winds W at 10 to 20 mph.HumidityHumidity89%UV LevelUV Index1 of 10Sun RiseSunrise7:13 amSunsetSunset4:50 pmWed 08 | Night47°CloudyRain18%WindWNW 17 mphOvercast. Low 47F. Winds WNW at 10 to 20 mph.HumidityHumidity79%UV LevelUV Index0 of 10Moon RiseMoonrise11:37 amMoo

In [8]:
ten_day_list = ['Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday','Monday','Tuesday','Wednesday','Thursday']
dates_list = pd.date_range('2021-12-07','2021-12-16')

string = ''
list = []
for i in range(len(text)):
    if ord(text[i]) != 124:
        string = string + text[i]
    else:
        if string != '':
            list.append(string.strip())
            string = ''
else:
    list.append(string.strip())
data = list[1:21]

# Separating temperatures and descriptions
temperature_list = []
description_data = []
for i in range(len(data)):
    if i % 2 == 1:
        temperature_list.append(data[i][5:8])
        data[i] = data[i][8:]
    else:
        temperature_list.append(data[i][3:6])
        data[i] = data[i][6:]


def celsius_converter(farhenheit):
    celsius = (int(farhenheit[:2])-32)*(5/9)
    return f'{int(celsius)}\N{DEGREE SIGN}'

# converting temps to celsius
highs_celsius = []
lows_celsius = []
for i in range(len(temperature_list)):
    if i % 2 == 1:
        lows_celsius.append(celsius_converter(temperature_list[i]))
    else:
        highs_celsius.append(celsius_converter(temperature_list[i]))
        

# cleaning up description
for i in range(len(data)):
    for j in range(len(data[i])):
        # adds spaces between letters and numbers
        if 96<ord(data[i][j-1])<123 and 47<ord(data[i][j])<58:
            data[i] = data[i][:j] + ' ' + data[i][j:]
        # adds spaces between words where needed
        if 96<ord(data[i][j-1])<123 and 64<ord(data[i][j])<91:
            data[i] = data[i][:j] + ' ' + data[i][j:]
        # adds spaces after % signs
        if ord(data[i][j]) == 37:
            data[i] = data[i][:j+1] + ' ' + data[i][j+1:]
        if 96<ord(data[i][j-1].lower())<123 and 47<ord(data[i][j])<58:
             data[i] = data[i][:j] + ' ' + data[i][j:]
        

#day and night descriptions
day_desc = []
night_desc = []
for i in range(len(data)):
    if i % 2 == 1:
        night_desc.append(data[i])
    else:
        day_desc.append(data[i])


#DataFrame
forecast = pd.DataFrame({
    'Day of the Week':ten_day_list,
    'Date':dates_list,
    'Daytime':day_desc,
    'Nighttime':night_desc,
    'High':highs_celsius,
    'Low':lows_celsius
})

forecast

,Day of the Week,Date,Daytime,Nighttime,High,Low
0,Tuesday,2021-12-07,Partly Cloudy Rain 7% Wind N 9 mph Partly clou...,Mostly Cloudy Night Rain 17% Wind W 5 mph Part...,14°,10°
1,Wednesday,2021-12-08,Cloudy Rain 21% Wind W 13 mph Overcast. High a...,Cloudy Rain 18% Wind WNW 17 mph Overcast. Low ...,12°,8°
2,Thursday,2021-12-09,Sunny Rain 3% Wind NW 14 mph Sunny skies. High...,Mostly Clear Night Rain 2% Wind NNW 15 mph Par...,12°,6°
3,Friday,2021-12-10,Sunny Rain 2% Wind NNW 6 mph Abundant sunshine...,Partly Cloudy Night Rain 5% Wind SE 5 mph Clea...,12°,5°
4,Saturday,2021-12-11,Mostly Sunny Rain 5% Wind S 7 mph Generally su...,Scattered Showers Night Rain 60% Wind SSE 10 m...,12°,7°
5,Sunday,2021-12-12,Scattered Showers Rain 81% Wind SSE 15 mph Clo...,Rain Rain 80% Wind S 15 mph Rain. Low 49F. Win...,12°,9°
6,Monday,2021-12-13,Rain Rain 75% Wind SSW 19 mph Periods of rain....,Rain Rain 70% Wind SW 17 mph Rain. Low 43F. Wi...,12°,6°
7,Tuesday,2021-12-14,Scattered Showers Rain 37% Wind W 11 mph Rain ...,Scattered Showers Night Rain 37% Wind SW 10 mp...,9°,5°
8,Wednesday,2021-12-15,Scattered Showers Rain 33% Wind S 9 mph Cloudy...,Scattered Showers Rain 32% Wind SSE 8 mph Clou...,11°,5°
9,Thursday,2021-12-16,Scattered Showers Rain 33% Wind SSE 8 mph A mi...,Scattered Showers Rain 35% Wind SSE 7 mph Over...,11°,5°
